### Run UK Wide

In [1]:
# Import modules etc
from helper_functions import expand_bbox, create_bounding_box, create_graph, get_edges_from_geom, assign_weights, compute_pareto_fronts, dict_to_array, generate_reference_point, hypervolume, spread_diversity, compute_signed_area, compute_trade_off_rate, haversine_distance
import psycopg2
import pyproj
from dotenv import load_dotenv
import os
import geopandas as gpd
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import math
from shapely.geometry import box, Point
import networkx as nx
import pickle


def calculate_haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate Haversine distance between two points in kilometers."""
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

def calculate_haversine_bounding_box(point, distance_km):
    """
    Calculate a bounding box using Haversine distance around a Point (lat, lon) extended by the given distance.

    Args:
        point: A geopy Point object with latitude and longitude.
        distance_km: Distance to extend the bounding box in kilometers.

    Returns:
        A dictionary representing the bounding box with 'min_lat', 'max_lat', 'min_lon', and 'max_lon'.
    """
    # Haversine formula uses Earth's radius in kilometers
    earth_radius_km = 6371.0

    # Convert the distance to radians (distance / Earth's radius)
    distance_rad = distance_km / earth_radius_km

    # Latitude and longitude in radians
    lat_rad = math.radians(point.centroid.y)
    lon_rad = math.radians(point.centroid.x)

    # Calculate latitude bounds
    min_lat = math.degrees(lat_rad - distance_rad)
    max_lat = math.degrees(lat_rad + distance_rad)

    # Calculate longitude bounds
    # Longitude depends on latitude, so adjust accordingly
    min_lon = math.degrees(lon_rad - distance_rad / math.cos(lat_rad))
    max_lon = math.degrees(lon_rad + distance_rad / math.cos(lat_rad))

    return {
        'min_lat': min_lat,
        'max_lat': max_lat,
        'min_lon': min_lon,
        'max_lon': max_lon
    }
    
def sample_dataframe(df, min_samples ,rate=0.1):
    """
    Samples rows from the input DataFrame based on a variable sampling rate.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    rate (float): The sampling rate for DataFrames with more than 10 rows.
    
    Returns:
    pd.DataFrame: The sampled DataFrame.
    """
    num_rows = len(df)
    
    if num_rows <= min_samples:
        # If the number of rows is 10 or less, return the entire DataFrame
        return df
    else:
        # Calculate the sample size
        sample_size = max(min_samples, int(num_rows * rate))
        
        # Sample the DataFrame
        sampled_df = df.sample(n=sample_size, random_state=1)
        return sampled_df

def mean_of_list(lst):
    return sum(lst)/len(lst)

In [2]:
def set_up_base_maps(lookup,next_lsoa,od_bike,msoas,con,project):
    
    #Get MSOA id assovciate to LSOA
    msoa_id = lookup[lookup['LSOA11CD'] == next_lsoa].iloc[0]['MSOA11CD']
    #Get MSOA OD pairs
    zone_od_flows = od_bike[od_bike['geo_code1'] == msoa_id]
    #Get MSOA with highest OD flow
    dest_msoa = zone_od_flows[zone_od_flows['bicycle'] == zone_od_flows['bicycle'].max()]['geo_code2'].values[0]
    #Get coords of origin centroid
    origin = (msoas.loc[msoa_id].centroid.y, msoas.loc[msoa_id].centroid.x)
    #Get coords of origin centroid
    destination = (msoas.loc[dest_msoa].centroid.y, msoas.loc[dest_msoa].centroid.x)
    #Calculate distance between coords
    distance_to_d = calculate_haversine_distance(origin[0], origin[1], destination[0], destination[1])
    
    # Case where distnce to destination is 0 (same MSOA) get second most popular MSOA and get distance
    if distance_to_d == 0:
        dest_msoa = zone_od_flows[['geo_code2','bicycle']].sort_values('bicycle',ascending = False).iloc[1]['geo_code2']
        origin = (msoas.loc[msoa_id].centroid.y, msoas.loc[msoa_id].centroid.x)
        destination = (msoas.loc[dest_msoa].centroid.y, msoas.loc[dest_msoa].centroid.x)
        distance_to_d = calculate_haversine_distance(origin[0], origin[1], destination[0], destination[1])
    
    # Take distance as min of either 10km or dist to work dest
    distance = min([15,distance_to_d])
    
    # Calcualted boiunding box add small buffer to account for routing around origin and destination points
    box_dist = distance * 1.25
    bbox = calculate_haversine_bounding_box(msoas.loc[msoa_id].centroid, box_dist)

    base_networks = {}
    base_networks[2016] = {}
    base_networks[2021] = {}
    #Get base networks for 2016 and 2021
    od_edges_2016 = get_edges_from_geom(box(*(bbox['min_lon'],bbox['min_lat'],bbox['max_lon'],bbox['max_lat'])), con, project, 'rn2016_roads')
    G_od_2016, nodes_od_2016, edges_od_2016 = create_graph(od_edges_2016)
    base_networks[2016]['G'] = G_od_2016
    base_networks[2016]['nodes'] = nodes_od_2016
    base_networks[2016]['edges'] = edges_od_2016
    
    od_edges_2021 = get_edges_from_geom(box(*(bbox['min_lon'],bbox['min_lat'],bbox['max_lon'],bbox['max_lat'])), con, project, 'rn2021_roads')
    G_od_2021, nodes_od_2021, edges_od_2021 = create_graph(od_edges_2021)
    base_networks[2021]['G'] = G_od_2021
    base_networks[2021]['nodes'] = nodes_od_2021
    base_networks[2021]['edges'] = edges_od_2021
    
    return base_networks, bbox

def coarsen_network(base_graph_nodes, base_graph, bbox, lsoa_centroids, next_lsoa, lsoas):
    
    # Get LSOAs intersecting with bbox
    od_polygon = gpd.GeoSeries([box(bbox['min_lon'],bbox['min_lat'],bbox['max_lon'],bbox['max_lat'])])
    lsoas_in_area = lsoa_centroids[lsoa_centroids.intersects(od_polygon.unary_union)]
    
    # Network nodes as points
    network_node_points_list = []
    for i,r in base_graph_nodes.iterrows():
        network_node_points_list.append(Point([r['latitude'],r['longitude']]))
    base_graph_nodes['geometry'] = network_node_points_list
    base_graph_nodes = gpd.GeoDataFrame(base_graph_nodes, geometry = 'geometry')

    # Associate lsoa to graph nodes (e.g., intersect graph nodes with lsoa boundary)
    area_to_nodes = {}
    num_nodes_per_oa = []
    for lsoa in list(lsoas_in_area.index):
        oa_geom = gpd.GeoSeries([lsoas.loc[lsoa]['geometry']])
        oa_nodes = base_graph_nodes[base_graph_nodes.intersects(oa_geom.unary_union)]
        area_to_nodes[lsoa] = oa_nodes
        num_nodes_per_oa.append(len(oa_nodes))
    
    #Create new network in order of LTS
    #LTS1
    lts_1_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) == 1]
    G_lts1 = nx.Graph()
    G_lts1.add_nodes_from(base_graph.nodes(data = True))
    G_lts1.add_edges_from(lts_1_edges)

    #LTS2
    lts_2_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if 0 < d.get('lts', 0) <= 2]
    G_lts2 = nx.Graph()
    G_lts2.add_nodes_from(base_graph.nodes(data = True))
    G_lts2.add_edges_from(lts_2_edges)

    #LTS3
    lts_3_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if 0 < d.get('lts', 0) <= 3]
    G_lts3 = nx.Graph()
    G_lts3.add_nodes_from(base_graph.nodes(data = True))
    G_lts3.add_edges_from(lts_3_edges)

    #LTS4
    lts_4_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if 0 < d.get('lts', 0) <= 4]
    G_lts4 = nx.Graph()
    G_lts4.add_nodes_from(base_graph.nodes(data = True))
    G_lts4.add_edges_from(lts_4_edges)
    
    #Get list of LSOA indeces
    lsoas_in_area_list = list(lsoas_in_area.index)

    
    paths_lts_1 = {}
    paths_lts_2 = {}
    paths_lts_3 = {}
    paths_lts_4 = {}

    paths_lts_1[next_lsoa] = {}
    paths_lts_2[next_lsoa] = {}
    paths_lts_3[next_lsoa] = {}
    paths_lts_4[next_lsoa] = {}
    
    for d in lsoas_in_area_list:
        paths_lts_1[next_lsoa][d] = []
        paths_lts_2[next_lsoa][d] = []
        paths_lts_3[next_lsoa][d] = []
        paths_lts_4[next_lsoa][d] = []
    

    origin_nodes = sample_dataframe(area_to_nodes[next_lsoa],4,0.1)
    for next_o_node in list(origin_nodes['node']):
        all_sp_lts1 = nx.single_source_dijkstra_path_length(G_lts1,next_o_node, weight = 'length')
        all_sp_lts2 = nx.single_source_dijkstra_path_length(G_lts2,next_o_node, weight = 'length')
        all_sp_lts3 = nx.single_source_dijkstra_path_length(G_lts3,next_o_node, weight = 'length')
        all_sp_lts4 = nx.single_source_dijkstra_path_length(G_lts4,next_o_node, weight = 'length')
        
        for d in lsoas_in_area_list:
            
            destination_nodes = list(area_to_nodes[d]['node'])

            #Add LTS 1 Routes to Dict
            paths_to_dest = {key: all_sp_lts1[key] for key in destination_nodes if key in all_sp_lts1}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_1[next_lsoa][d].extend(paths_to_dest)

            #Add LTS 2 Routes to Dict
            paths_to_dest = {key: all_sp_lts2[key] for key in destination_nodes if key in all_sp_lts2}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_2[next_lsoa][d].extend(paths_to_dest)
            
            #Add LTS 3 Routes to Dict
            paths_to_dest = {key: all_sp_lts3[key] for key in destination_nodes if key in all_sp_lts3}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_3[next_lsoa][d].extend(paths_to_dest)
            
            #Add LTS 4 Routes to Dict
            paths_to_dest = {key: all_sp_lts4[key] for key in destination_nodes if key in all_sp_lts4}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            if len(paths_to_dest) > 0:
                #Add found routes to dictionary
                paths_lts_4[next_lsoa][d].extend(paths_to_dest)
                
    dist_dict = {}
    
    for d in lsoas_in_area_list:
        distances = []
        if len(paths_lts_1[next_lsoa][d]) > 0:
            distances.append(mean_of_list(paths_lts_1[next_lsoa][d]))
        else:
            distances.append(0)

        if len(paths_lts_2[next_lsoa][d]) > 0:
            distances.append(mean_of_list(paths_lts_2[next_lsoa][d]))
        else:
            distances.append(0)

        if len(paths_lts_3[next_lsoa][d]) > 0:
            distances.append(mean_of_list(paths_lts_3[next_lsoa][d]))
        else:
            distances.append(0)
            
        if len(paths_lts_4[next_lsoa][d]) > 0:
            distances.append(mean_of_list(paths_lts_4[next_lsoa][d]))
        else:
            distances.append(0)
        
        dist_dict[d] = distances
        
    return dist_dict

In [3]:
# Get DB username and password from local .env file
load_dotenv()
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

#Connect to DB
con = psycopg2.connect(database="ukrn", user=db_user, password=db_password, host="/var/run/postgresql/")
project = pyproj.Transformer.from_proj(pyproj.Proj(init='epsg:4326'),pyproj.Proj(init='epsg:3857'))
project_reverse = pyproj.Transformer.from_proj(pyproj.Proj(init='epsg:3857'),pyproj.Proj(init='epsg:4326'))

/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferre

In [4]:
# For 2021 create UK wide network as base network

# Import MSOA lookup
msoas = gpd.read_file('data/MSOA_EngWal_Dec_2011_Generalised_ClippedEW_0/Middle_Layer_Super_Output_Areas_December_2011_Generalised_Clipped_Boundaries_in_England_and_Wales.shp').to_crs(4326).set_index('msoa11cd')

# Import MSOA 2011 OD data
od_data = pd.read_parquet('data/od_2011.parquet')

#Import LSOAs
lsoas = gpd.read_file('data/LSOA_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V4_6029841263726194941.gpkg').to_crs(4326)
lsoas = lsoas[lsoas['LSOA11CD'].str.startswith('E')]
lsoas = pd.concat([lsoas, lsoas.bounds], axis=1)

#Import lsoa to msoa look up
lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')
lsoas_ids = list(lsoas['LSOA11CD'])
lsoas = lsoas.set_index('LSOA11CD')

lsoa_centroids = []
for i,r in lsoas.iterrows():
    lsoa_centroids.append(r['geometry'].centroid)
lsoas['centroid'] = lsoa_centroids
lsoa_centroids = gpd.GeoDataFrame(lsoas[['centroid']], geometry='centroid')

lsoas_processed = pd.read_csv('data/lsoas_processed.csv', index_col = 0)

centroids = []
for _i, _r in msoas.iterrows():
    centroids.append(_r['geometry'].centroid)
msoas['centroid'] = centroids

/tmp/ipykernel_1925211/1743920208.py:15: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')


In [5]:
od_bike = od_data[od_data['bicycle'] > 0]
od_bike = od_bike[od_bike['geo_code1'].str.startswith('E')]
od_bike = od_bike[od_bike['geo_code2'].str.startswith('E')]

In [12]:
coarse_edges_2016 = {}
coarse_edges_2021 = {}

In [13]:
lsoa_sample = list(lsoas_processed[~lsoas_processed['processed']].sample(2500).index)
num_processed = 0

for next_lsoa in lsoa_sample:
    
    num_processed += 1
    print('Num processed : {}'.format(num_processed))

    base_networks, bbox = set_up_base_maps(lookup,next_lsoa,od_bike,msoas,con,project)

    year = 2016
    coarse_edges_2016[next_lsoa] = coarsen_network(base_networks[year]['nodes'], base_networks[year]['G'], bbox, lsoa_centroids, next_lsoa, lsoas)
    year = 2021
    coarse_edges_2021[next_lsoa] = coarsen_network(base_networks[year]['nodes'], base_networks[year]['G'], bbox, lsoa_centroids, next_lsoa, lsoas)

    with open('data/coarse_edges_2016.pkl', 'wb') as f:
        pickle.dump(coarse_edges_2016, f)

    with open('data/coarse_edges_2021.pkl', 'wb') as f:
        pickle.dump(coarse_edges_2021, f)

    lsoas_processed.loc[next_lsoa,'processed'] = True
    lsoas_processed.to_csv('data/lsoas_processed.csv')

Num processed : 1


/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


Num processed : 2


/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


Num processed : 3


...

/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


Num processed : 118


/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


Num processed : 119


KeyError: 'E01018768'

In [24]:

for next_lsoa in lsoa_sample:
    num_processed += 1
    print('Number Processes : {}'.format(num_processed))
    msoa_id = lookup[lookup['LSOA11CD'] == next_lsoa].iloc[0]['MSOA11CD']
    zone_od = od_bike[(od_bike['geo_code1'] == msoa_id)][['geo_code1','geo_code2']]
    zone_od_flows = od_bike[od_bike['geo_code1'] == msoa_id]

    dest_msoa = zone_od_flows[zone_od_flows['bicycle'] == zone_od_flows['bicycle'].max()]['geo_code2'].values[0]
    origin = (msoas.loc[msoa_id].centroid.y, msoas.loc[msoa_id].centroid.x)
    destination = (msoas.loc[dest_msoa].centroid.y, msoas.loc[dest_msoa].centroid.x)
    distance = calculate_haversine_distance(origin[0], origin[1], destination[0], destination[1])
    print(distance)

    # Case where 0 get second most popular MSOA and get distance
    if distance == 0:
        dest_msoa = zone_od_flows[['geo_code2','bicycle']].sort_values('bicycle',ascending = False).iloc[1]['geo_code2']
        origin = (msoas.loc[msoa_id].centroid.y, msoas.loc[msoa_id].centroid.x)
        destination = (msoas.loc[dest_msoa].centroid.y, msoas.loc[dest_msoa].centroid.x)
        distance_to_dest = calculate_haversine_distance(origin[0], origin[1], destination[0], destination[1])
        distance = min([15,distance_to_dest])
        print(distance)
        
    box_dist = distance * 1.25


    bbox = calculate_haversine_bounding_box(msoas.loc[msoa_id].centroid, box_dist)

    od_edges_2016 = get_edges_from_geom(box(*(bbox['min_lon'],bbox['min_lat'],bbox['max_lon'],bbox['max_lat'])), con, project, 'rn2016_roads')
    G_od_2016, nodes_od_2016, edges_od_2016 = create_graph(od_edges_2016)

    od_edges_2021 = get_edges_from_geom(box(*(bbox['min_lon'],bbox['min_lat'],bbox['max_lon'],bbox['max_lat'])), con, project, 'rn2021_roads')
    G_od_2021, nodes_od_2021, edges_od_2021 = create_graph(od_edges_2021)
    
    base_graph_nodes = nodes_od_2021
    base_graph = G_od_2021

    od_polygon = gpd.GeoSeries([box(bbox['min_lon'],bbox['min_lat'],bbox['max_lon'],bbox['max_lat'])])
    oas_in_area = lsoa_centroids[lsoa_centroids.intersects(od_polygon.unary_union)]

    # Network nodes as points
    network_node_points_list = []
    for i,r in base_graph_nodes.iterrows():
        network_node_points_list.append(Point([r['latitude'],r['longitude']]))

    base_graph_nodes['geometry'] = network_node_points_list
    base_graph_nodes = gpd.GeoDataFrame(base_graph_nodes, geometry = 'geometry')

    area_to_nodes = {}
    num_nodes_per_oa = []
    for oa in list(oas_in_area.index):
        oa_geom = gpd.GeoSeries([lsoas.loc[oa]['geometry']])
        oa_nodes = base_graph_nodes[base_graph_nodes.intersects(oa_geom.unary_union)]
        area_to_nodes[oa] = oa_nodes
        num_nodes_per_oa.append(len(oa_nodes))
        
    #LTS1
    lts_1_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) <= 1]
    G_lts1 = nx.Graph()
    G_lts1.add_nodes_from(base_graph.nodes(data = True))
    G_lts1.add_edges_from(lts_1_edges)

    #LTS2
    lts_2_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) <= 2]
    G_lts2 = nx.Graph()
    G_lts2.add_nodes_from(base_graph.nodes(data = True))
    G_lts2.add_edges_from(lts_2_edges)

    #LTS3
    lts_3_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) <= 3]
    G_lts3 = nx.Graph()
    G_lts3.add_nodes_from(base_graph.nodes(data = True))
    G_lts3.add_edges_from(lts_3_edges)

    #LTS4
    lts_4_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) <= 4]
    G_lts4 = nx.Graph()
    G_lts4.add_nodes_from(base_graph.nodes(data = True))
    G_lts4.add_edges_from(lts_4_edges)

    oas_in_area_list = list(oas_in_area.index)

    paths_lts_1 = {}
    paths_lts_2 = {}
    paths_lts_3 = {}
    paths_lts_4 = {}
    operations = 0

    for o in oas_in_area_list:
        paths_lts_1[o] = {}
        paths_lts_2[o] = {}
        paths_lts_3[o] = {}
        paths_lts_4[o] = {}
        for d in oas_in_area_list:
            operations += 1
            paths_lts_1[o][d] = []
            paths_lts_2[o][d] = []
            paths_lts_3[o][d] = []
            paths_lts_4[o][d] = []
            
    # Define the original layers as MultiGraphs

    o = next_lsoa
    origin_nodes = sample_dataframe(area_to_nodes[o],4,0.1)
    for next_o_node in list(origin_nodes['node']):
        all_sp_lts1 = nx.single_source_dijkstra_path_length(G_lts1,next_o_node, weight = 'length')
        operations += 1
        all_sp_lts2 = nx.single_source_dijkstra_path_length(G_lts2,next_o_node, weight = 'length')
        operations += 1
        all_sp_lts3 = nx.single_source_dijkstra_path_length(G_lts3,next_o_node, weight = 'length')
        operations += 1
        all_sp_lts4 = nx.single_source_dijkstra_path_length(G_lts4,next_o_node, weight = 'length')
        operations += 1
        
        for d in oas_in_area_list:
        
            operations += 1
            
            destination_nodes = list(area_to_nodes[d]['node'])

            #Add LTS 1 Routes to Dict
            paths_to_dest = {key: all_sp_lts1[key] for key in destination_nodes if key in all_sp_lts1}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_1[o][d].extend(paths_to_dest)

            #Add LTS 2 Routes to Dict
            paths_to_dest = {key: all_sp_lts2[key] for key in destination_nodes if key in all_sp_lts2}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_2[o][d].extend(paths_to_dest)
            
            #Add LTS 3 Routes to Dict
            paths_to_dest = {key: all_sp_lts3[key] for key in destination_nodes if key in all_sp_lts3}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_3[o][d].extend(paths_to_dest)
            
            #Add LTS 4 Routes to Dict
            paths_to_dest = {key: all_sp_lts4[key] for key in destination_nodes if key in all_sp_lts4}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            if len(paths_to_dest) > 0:
                #Add found routes to dictionary
                paths_lts_4[o][d].extend(paths_to_dest)
                
    dist_dict = {}

    o = next_lsoa
    for d in oas_in_area_list:
        distances = []
        if len(paths_lts_1[o][d]) > 0:
            distances.append(mean_of_list(paths_lts_1[o][d]))
        else:
            distances.append(0)

        if len(paths_lts_2[o][d]) > 0:
            distances.append(mean_of_list(paths_lts_2[o][d]))
        else:
            distances.append(0)

        if len(paths_lts_3[o][d]) > 0:
            distances.append(mean_of_list(paths_lts_3[o][d]))
        else:
            distances.append(0)
            
        if len(paths_lts_4[o][d]) > 0:
            distances.append(mean_of_list(paths_lts_4[o][d]))
        else:
            distances.append(0)
        
        dist_dict[d] = distances
        
    coarse_edges[o] = dist_dict

Number Processes : 1
2.1168556040430184


/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


Number Processes : 2
5.292231605998219


/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


Number Processes : 3
3.6058599250813113


...

Number Processes : 7
2.354781819358965


/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


Number Processes : 8
0.0
182.04495108731714


/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


KeyboardInterrupt: 